In [1]:
from torch_points3d.datasets.base_dataset import BaseDataset
from torch.utils.data import Dataset

In [2]:
import numpy as np
import pandas as pd
import time 
import os
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from torch_geometric.io import read_txt_array
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler

from plyfile import PlyData, PlyElement,PlyProperty, PlyListProperty

import sys
from omegaconf import OmegaConf
import pyvista as pv
import ipdb

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.max_colwidth", 256)

torch.set_printoptions(precision=8)

In [5]:
%matplotlib inline
sns.set(rc={'figure.figsize':(8,6)})


In [6]:
INPUT_PATH = os.getcwd()+'\\Save data\\Processed_data'
OUTPUT_TEMP_PATH = os.getcwd()+'\\Save data\\Processed_data\\Output_temp'

INPUT_TEMP_PATH = os.getcwd()+'\\Save data\\Processed_data\\Output_temp'
OUTPUT_PATH = os.getcwd()+'\\Save data\\Processed_data\\Output'

OUTPUT_PCA_PATH = os.getcwd()+'\\Save data\\Processed_data\\Output\\PCA'

In [7]:
patient_id_list_new =  '2_BC	3_BC	4_BC	5_BM	6_BM	7_BP	8_BP	9_KBW	10_SUM	11_DHM	12_GAW	13_PMM	14_TR	15_TR	16_TR	18_EM	19_EM	20_EM	21_FA	22_FA	23_HJ	24_HJ	25_HM	26_HM	27_HM	28_JM	29_JM	30_JM	31_JM	32_JM	33_KBB	34_KBB	35_KBB	36_KBB	37_KBB	38_KBB	39_KBB	40_KBB	41_KBB	42_KBB	43_KBB	44_AC	45_AC	46_LE	47_LE	48_LE	50_LE	51_LE	52_LE	54_MR	55_MR	56_WA	57_SF	58_SI	59_SI'
patient_id_list_new = patient_id_list_new.split()
patient_id_list_new[:5]

['2_BC', '3_BC', '4_BC', '5_BM', '6_BM']

In [8]:
ORIGINAL_FILENAME = "_PLY0.ply"
WSS_FILENAME = "_WSS.csv"
WSS_DOWN_FILENME = "_WSS_down.csv"
CURV_FILENAME = "_curv_down.csv"

PYMESH_FILENAME = "_pymesh.ply"
DESC_1_FILENAME = "_fpfh.ply"
DESC_2_FILENAME = "_shot.ply"

RADIUS_SEARCH = 0.00024
RADIUS = RADIUS_SEARCH * 0.6

OUTPUT_FILENAME = "_output.ply"
PCA_FILENAME = "_pca.ply"
os.path.join(OUTPUT_TEMP_PATH,f"{ORIGINAL_FILENAME}{PYMESH_FILENAME}")

'D:\\Workspace\\Python\\aneurysm-segmentation\\datasets\\Save data\\Processed_data\\Output_temp\\_PLY0.ply_pymesh.ply'

In [9]:
print (f"TOTAL FILES TO BE PROCESSED: {len(patient_id_list_new)}")

TOTAL FILES TO BE PROCESSED: 55


In [10]:
os.getcwd()

'D:\\Workspace\\Python\\aneurysm-segmentation\\datasets'

In [11]:
DIR = os.getcwd() # Replace with your root directory, the data will go in DIR/data.

In [12]:
#@title Configure the dataset {run: "auto"}
CATEGORY = "All" #@param ["Airplane", "Bag", "All", "Motorbike"] {allow-input: true}
USE_NORMALS = False #@param {type:"boolean"}|

In [17]:
shapenet_yaml = """
class: AneurysmDataset.AneurysmDataset
task: segmentation
dataroot: %s
first_subsampling: 0.02                       # Grid size of the input data
pre_transforms:                               # Offline transforms, done only once         
    - transform: NormalizeScale 
    - transform: GridSampling3D
      params:
        size: ${first_subsampling}
train_transforms:                             # Data augmentation pipeline
    - transform: RandomNoise
      params:
        sigma: 0.01
        clip: 0.05
    - transform: RandomScaleAnisotropic
      params:
        scales: [0.9,1.1]
    - transform: AddOnes
    - transform: AddFeatsByKeys
      params:
        list_add_to_x: [True]
        feat_names: ["ones"]
        delete_feats: [True]
test_transforms:
    - transform: AddOnes
    - transform: AddFeatsByKeys
      params:
        list_add_to_x: [True]
        feat_names: ["ones"]
        delete_feats: [True]
""" % (os.path.join(DIR,"data")) 

from omegaconf import OmegaConf
params = OmegaConf.create(shapenet_yaml)
# if CATEGORY != "All":
#     params.category = CATEGORY

In [12]:
# params.raw_file_identifiers = patient_id_list_new

In [13]:
params.raw_file_identifiers

['2_BC', '3_BC', '4_BC', '5_BM', '6_BM', '7_BP', '8_BP', '9_KBW', '10_SUM', '11_DHM', '12_GAW', '13_PMM', '14_TR', '15_TR', '16_TR', '18_EM', '19_EM', '20_EM', '21_FA', '22_FA', '23_HJ', '24_HJ', '25_HM', '26_HM', '27_HM', '28_JM', '29_JM', '30_JM', '31_JM', '32_JM', '33_KBB', '34_KBB', '35_KBB', '36_KBB', '37_KBB', '38_KBB', '39_KBB', '40_KBB', '41_KBB', '42_KBB', '43_KBB', '44_AC', '45_AC', '46_LE', '47_LE', '48_LE', '50_LE', '51_LE', '52_LE', '54_MR', '55_MR', '56_WA', '57_SF', '58_SI', '59_SI']

In [18]:
import AneurysmDataset
dataset = AneurysmDataset.AneurysmDataset(params)
dataset

Processing...


  0%|          | 0/2 [00:00<?, ?it/s]

loaded <_io.BufferedReader name='D:\\Workspace\\Python\\aneurysm-segmentation\\datasets\\data\\aneurysm\\raw\\4_BC_pca.ply'> 
loaded <_io.BufferedReader name='D:\\Workspace\\Python\\aneurysm-segmentation\\datasets\\data\\aneurysm\\raw\\5_BM_pca.ply'> 


  0%|          | 0/2 [00:00<?, ?it/s]

loaded <_io.BufferedReader name='D:\\Workspace\\Python\\aneurysm-segmentation\\datasets\\data\\aneurysm\\raw\\2_BC_pca.ply'> 
loaded <_io.BufferedReader name='D:\\Workspace\\Python\\aneurysm-segmentation\\datasets\\data\\aneurysm\\raw\\7_BP_pca.ply'> 


  0%|          | 0/2 [00:00<?, ?it/s]

loaded <_io.BufferedReader name='D:\\Workspace\\Python\\aneurysm-segmentation\\datasets\\data\\aneurysm\\raw\\14_TR_pca.ply'> 
loaded <_io.BufferedReader name='D:\\Workspace\\Python\\aneurysm-segmentation\\datasets\\data\\aneurysm\\raw\\18_EM_pca.ply'> 
Done!


Dataset: AneurysmDataset 
train_pre_batch_collate_transform = None
val_pre_batch_collate_transform = None
test_pre_batch_collate_transform = None
pre_transform = Compose([
    NormalizeScale(),
    GridSampling3D(grid_size=0.02, quantize_coords=False, mode=mean),
])
test_transform = Compose([
    AddOnes(),
    AddFeatsByKeys(ones=True),
])
train_transform = Compose([
    RandomNoise(sigma=0.01, clip=0.05),
    RandomScaleAnisotropic([0.9, 1.1]),
    AddOnes(),
    AddFeatsByKeys(ones=True),
])
val_transform = None
inference_transform = Compose([
    NormalizeScale(),
    GridSampling3D(grid_size=0.02, quantize_coords=False, mode=mean),
    AddOnes(),
    AddFeatsByKeys(ones=True),
])
Size of train_dataset = 2
Size of test_dataset = 2
Size of val_dataset = 2
Batch size = None